* Without Clause

Version Number | Tile | Level | Model
-------------- | ---- | ----- | -----
5 | Bangalore | L1C | Model V5-3 
6 | Jakarta | L1C | Model V5-3
7 | Bombay | L1C | Model V5-3
8 | Bangalore | L2A | Model V5-3
9 | Calais | L2A | Model V5-3
16 | Bangalore | L1C | Model V5-4
18 | Jakarta | L1C | Model V5-4
19 | Bombay | L1C | Model V5-4
20 | Delhi | L1C | Model V5-4
22 | Bombay | L1C | Model V5-5
23 | Jakarta | L1C | Model V5-5
24 | Mathura, Vizag | L1C | Model V5-5
25 | Bombay V2 | L1C | Model V5-5

* With Clause

Version Number | Tile | Model
-------------- | ---- | -----
10 | Bangalore | Model V5-3
11 | Bombay | Model V5-3
12 | Jakarta | Model V5-3

> Version 21 : Taking features of Misclassified SeaLake and comparing them to Forest Dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2, os, json, rasterio

from sklearn.preprocessing import *

from tensorflow.keras.models import load_model
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from tensorflow.keras.applications.resnet_v2 import preprocess_input

In [ ]:
def sliding_window(tif, stepSize, windowSize):
    
    for y in range(0, tif.shape[0], stepSize):
        for x in range(0, tif.shape[1], stepSize):
            
            yield(x,y, tif[y:y + windowSize[1], x:x+windowSize[0]])
            pass
        pass
    pass

In [ ]:
def Normalise(arr_band):
    
    arr_band = Normalizer().fit_transform(arr_band)
    return arr_band
    pass

In [ ]:
def preprocess_input(sub_tile):
    
    for i in range(0,6):
        sub_tile[:,:,i] = Normalise(sub_tile[:,:,i])
        pass
    
    return sub_tile
    pass

In [ ]:
loc_prob_df = pd.DataFrame(columns=["TileNumber","Class","Probability","AnnualCrop","Forest","HerbaceousVegetation","Highway","Industrial",
                                   "Pasture","PermanentCrop","Residential","River","SeaLake"])

In [ ]:
# Loading saved TensorFlow model
model = load_model("../input/eurosat-allbands-classification/6bands_v5-5.h5")
# Loading the best weights found while training
model.load_weights("../input/eurosat-allbands-classification/6bands_weights_v5-5.h5")

# Printing summary of the model
model.summary()

In [ ]:
bands = {'1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '8a':9, '9':10, '10':11, '11':12, '12':13}
bands_L2A = {'1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '8a':9, '9':10, '11':11, '12':12}

In [ ]:
src = rasterio.open("../input/testdata/BombayV2-Merged.tif")

In [ ]:
%%time
arr_3, arr_4, arr_8 = src.read(bands['3']), src.read(bands['4']), src.read(bands['8'])
arr_6, arr_7 = src.read(bands['6']), src.read(bands['7'])
arr_11 = src.read(bands['11'])

arr_3 = np.array(arr_3, dtype=np.float32)
arr_4 = np.array(arr_4, dtype=np.float32)
arr_6, arr_7 = np.array(arr_6, dtype=np.float32), np.array(arr_7, dtype=np.float32)
arr_8 = np.array(arr_8, dtype=np.float32)
arr_11 = np.array(arr_11, dtype=np.float32)

In [ ]:
%%time
bands_10_20 = np.dstack((arr_3, arr_4, arr_6, arr_7, arr_8, arr_11))

bands_10_20.shape

In [ ]:
test_img = cv2.imread("../input/testdata/TCI-BombayV2.png")

In [ ]:
with open("../input/testdata/label_map.json","r") as f:
    class_names = json.load(f)
    pass

class_names = {i:k for k,i in class_names.items()}
class_count = {k:0 for i,k in class_names.items()}
classes = list(class_names.values())

print("Class names and their labels : {}".format(class_names))
print("Class Names : {}".format(classes))
print("Classes and the number of times they have been detected : {}".format(class_count))

# Classifying sub-tiles within the tile

In [ ]:
%%time

c = 0

for (x,y, roiOrig) in sliding_window(bands_10_20, 64, (64,64)):
    
    roi = preprocess_input(roiOrig)
    roi = cv2.resize(roiOrig, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
    roi = np.expand_dims(roi, axis=0)
    # print(roi.shape)

    pred_value = model.predict(roi) # predicting the land use
    pred_prob = np.max(pred_value)
    pred_name = class_names[np.argmax(pred_value)]
    class_count[pred_name] += 1
    # print(pred_value)
    
    cv2.rectangle(test_img, (x,y), (x+64,y+64), (0,255,0),2)
    label = "{}".format(pred_name)
    prob = "{:.2f}%".format(pred_prob*100)
    label_bin = "{}".format(np.argmax(pred_value))

    # Storing tile number and predicted class label
    tileNumber = "{}".format(c+1)
    cv2.putText(test_img, tileNumber, (x+2,y+15), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0,0,255), 1)
    cv2.putText(test_img, label_bin, (x+2,(2*y+64)//2), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,0), 1)
    
    label_prob = "{} {}".format(label, prob)

    loc_prob_df = loc_prob_df.append({'TileNumber':tileNumber,'Class':label,'Probability':prob,
                                      "AnnualCrop":pred_value[0][0],"Forest":pred_value[0][1],
                                      "HerbaceousVegetation":pred_value[0][2],"Highway":pred_value[0][3],
                                      "Industrial":pred_value[0][4],"Pasture":pred_value[0][5],
                                      "PermanentCrop":pred_value[0][6],"Residential":pred_value[0][7],
                                      "River":pred_value[0][8],"SeaLake":pred_value[0][9]}, ignore_index=True)

    c += 1
    print("\r{}".format(c),end="")
    pass

print()

In [ ]:
loc_prob_df.shape

In [ ]:
print("[INFO] Saving image with bounding box, class label and tile number")
cv2.imwrite("Model-V5_5-BombayV2.png",test_img)

print(class_count)

print("[INFO] Saving results to a CSV file")
loc_prob_df.to_csv("Model-V5_5-BombayV2.csv", index=False)